<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/miltonAnimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install cartopy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cft
import pandas as pd
from datetime import datetime as dt

Set user parameters

In [ ]:
name = "Milton"
tcNum = "14"
trackType = ""

initTime = dt(year=2024, month=10, day=8, hour=6)
initStr, initHour = initTime.strftime("%Y%m%d_%H").split("_")

fHourStep = 3      #Normally 3 for HAFS-A
forecastLength = 48 #Normally 126 for HAFS-A.
#runStep = 6         #Normally 6 for HAFS-A

figureSuffix = "_RI"
subfolder = "RI/"
dataPath = "/content/drive/MyDrive/savedData/"
figurePath = "/content/drive/MyDrive/figures/"

potentialTemp = True #Use atmospheric potential temperature instead of in-situ temperature
stormCentered = True

lonMin, lonMax, latMin, latMax = -92, -85, 21, 25 #used if stormCetered = False
mapRadius = 0.75
atmLayer = 1000.0

Find times needed

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

#Times to plot
fcastTimes = [] #Key: initiation, item: valid time list
fhour = 0
validTime = initTime
while fhour <= forecastLength:
    fcastTimes.append(validTime)
    validTime += pd.Timedelta(hours=fHourStep)
    fhour += fHourStep

Find TC Location

In [ ]:
cols = ["BASIN", "CY", "YYYYMMDDHH", "TECHNUM/MIN", "TECH", "TAU", "LatN/S", "LonE/W",
    "VMAX", "MSLP", "TY", "RAD", "WINDCODE", "RAD1", "RAD2", "RAD3", "RAD4",
    "POUTER", "ROUTER", "RMW", "GUSTS", "EYE", "SUBREGION", "MAXSEAS", "INITIALS",
    "DIR", "SPEED", "STORMNAME", "DEPTH", "SEAS", "SEASCODE", "SEAS1", "SEAS2",
    "SEAS3", "SEAS4", "USERDEFINED1", "Thermo1", "Thermo2", "Thermo3", "Thermo4",
    "Thermo5", "Thermo6", "Thermo7", "USERDEFINED2", "DT", "SHR82", "SHR81_1",
    "SHR82_2",  "USERDEFINED3", "SST", "USERDEFINED4", "ARMW1", "ARMW2"]

initStr, initHour = initTime.strftime("%Y%m%d_%H").split("_")

atcfURL = f"https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/{initStr}/{initHour}/{tcNum}l.{initStr}{initHour}.hfsa.trak.atcfunix"
atcfFile = "atcf_" + initStr + "_" + initHour + ".csv"

!wget -O {atcfFile} {atcfURL}
atcf = pd.read_csv(atcfFile, names=cols)

In [ ]:
tcLocs = {}
rmws = {}
vmax = {}

for valid in fcastTimes:
    fHour = int((valid-initTime).total_seconds() / 3600)

    pointLat = int(atcf[atcf.TAU==fHour]["LatN/S"].iloc[0].replace("N", ""))/10
    pointLon = int(atcf[atcf.TAU==fHour]["LonE/W"].iloc[0].replace("W", ""))/-10 #Assume western hemisphere
    tcLocs[valid] = (pointLat, pointLon)

    rmws[valid] = atcf[atcf.TAU==fHour]["RMW"].iloc[0] * 1.852 #n mi to km
    vmax[valid] = atcf[atcf.TAU==fHour]["VMAX"].iloc[0]


Create animation

In [ ]:
atmFig, atmAx = plt.subplots(figsize=(10, 10), subplot_kw={"projection": ccrs.PlateCarree()})
contourLevs = np.linspace(0, 80, 17)

def update(valid):
  global lonMin, lonMax, latMin, latMax

  fhour = str(int((valid-initTime).total_seconds() / 3600))
  while len(fhour) < 3: fhour = "0" + fhour

  atmFile = "hafsa_" + initStr + initHour + "_f" + fhour + ".nc"
  atmPath = dataPath + "hafsaOutput/" + subfolder + atmFile
  atmData = xr.open_dataset(atmPath)

  if stormCentered: lonMin, lonMax, latMin, latMax = tcLocs[valid][1]-mapRadius, tcLocs[valid][1]+mapRadius, tcLocs[valid][0]-mapRadius, tcLocs[valid][0]+mapRadius
  atmSlice = atmData.sel(isobaricInhPa=atmLayer).sel(longitude=slice(lonMin+360, lonMax+360), latitude=slice(latMin, latMax))
  dat = np.sqrt(atmSlice.u.data ** 2 + atmSlice.v.data ** 2)

  tempContour = atmAx.contourf(atmSlice.longitude.data, atmSlice.latitude.data, dat, cmap="viridis", transform=ccrs.PlateCarree(), levels=contourLevs)
                              #extent = [lonMin, lonMax, latMin, latMax], levels=contourLevs)
  atmAx.coastlines()
  atmAx.gridlines(draw_labels=["left", "bottom"], alpha=0.5)
  atmAx.set_title(valid.strftime("%Y-%m-%d %HUTC"))

  atmAx.set_extent([lonMin, lonMax, latMin, latMax])

  return [tempContour] # Return a list of artists

def init():
  tempContour = update(fcastTimes[0])
  atmFig.colorbar(tempContour[0], shrink=0.4, label="Wind Speed (ms$^{-1}$)")
  return tempContour

ani = FuncAnimation(atmFig, update, frames=fcastTimes,
                    init_func=init, blit=True)
ani.save(f"miltonWind_{initTime.strftime('%m%d%H')}.gif", fps=1.5)
plt.show()

RMW with time

In [ ]:
rmwAx.xticksparams(rotation=45)

In [ ]:
fig = plt.figure(figsize=(8, 4))
rmwAx = fig.add_subplot()
vmaxAx = rmwAx.twinx()

rmwAx.plot(rmws.keys(), rmws.values(), color="b")
vmaxAx.plot(vmax.keys(), vmax.values(), color="r")

rmwAx.set_ylabel("RMW (km)", color="b")
vmaxAx.set_ylabel("Vmax (kt)", color="r")
rmwAx.grid(alpha=0.5)
fig.suptitle(initTime.strftime("%Y-%m-%d %HUTC"))